In [1]:
import os, sys
import django
import pytz
from dateutil import parser
from django.utils.timezone import make_aware
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "esr21.settings"

# Connect to Django ORM
django.setup()

In [4]:
idxs = VaccinationHistory.objects.filter(
    dose_quantity='1', dose1_product_name='janssen').values_list('subject_identifier', flat=True)
booster_doses = VaccinationDetails.objects.filter(site_id=40,
    received_dose_before='booster_dose', subject_visit__subject_identifier__in=idxs).values_list('subject_visit__subject_identifier', flat=True)
booster_doses

<QuerySet ['150-040993793-3', '150-040993788-3', '150-040993833-7', '150-040993774-3', '150-040993807-1', '150-040993826-1', '150-040993790-9']>

In [5]:
from esr21_subject.helper_classes import EnrollmentHelper
from edc_appointment.constants import NEW_APPT
subject_idxs = booster_doses
for idx in subject_idxs:
    try:
        VaccinationHistory.objects.get(subject_identifier=idx)
    except VaccinationHistory.DoesNotExist:
        print('vaccination history missing for, ', idx)
    else:
        # Change existing schedule name **onschedule and history obj
        # Consideration for cohort ??
        onschedule = OnSchedule.objects.filter(subject_identifier=idx)
        
        cohort = onschedule.first().schedule_name.replace('_boost_schedule', '')
        old_schedule_name = onschedule.first().schedule_name
        placeholder_name = f'{old_schedule_name}_old'

        onschedule.update(schedule_name=placeholder_name)
        SubjectScheduleHistory.objects.filter(subject_identifier=idx).update(schedule_name=placeholder_name)

        # Update appts and switch completed visits
        appts = Appointment.objects.filter(subject_identifier=idx).exclude(appt_status=NEW_APPT)
        print(appts)
        
        # Put subject on correct schedules
        helper = EnrollmentHelper(cohort=cohort, subject_identifier=idx)
        helper.schedule_enrol()

        # Update second dose appointments with data from booster appointments
        for appt in appts:
            rel_code = None
            if appt.visit_code == '1170':
                rel_code = '1070'
            elif appt.visit_code == '1198':
                rel_code = '1098'
    
            new_appt = Appointment.objects.filter(
                subject_identifier=idx, visit_code=rel_code, visit_code_sequence=appt.visit_code_sequence)
            new_appt.update(
                appt_datetime=appt.appt_datetime,
                appt_type=appt.appt_type,
                appt_status=appt.appt_status,
                appt_reason=appt.appt_reason,
                comment=appt.comment)

            # Update related subject visits
            try:
                visit = SubjectVisit.objects.get(appointment=appt)
            except SubjectVisit.DoesNotExist:
                pass
            else:
                # Update related appt to new_appt, and schedule name.
                visit.appointment=new_appt[0]
                visit.schedule_name=new_appt[0].schedule_name
                visit.save()

                # Update vaccination details, received dose 'booster' --> 'second' dose.
                vac_details = VaccinationDetails.objects.filter(subject_visit=visit)
                if vac_details:
                    vac_details.update(received_dose_before = 'second_dose')
        
            # Reset old appointments
            appt.appt_status = NEW_APPT
            appt.appt_reason = ''
            appt.save()

        # Delete old onschedules, and history
        OnSchedule.objects.filter(subject_identifier=idx, schedule_name=placeholder_name).delete()
        SubjectScheduleHistory.objects.filter(subject_identifier=idx, schedule_name=placeholder_name).delete()

<QuerySet [<Appointment: 1170.0>]>
<QuerySet [<Appointment: 1170.0>]>
<QuerySet [<Appointment: 1170.0>]>
<QuerySet [<Appointment: 1170.0>]>
<QuerySet [<Appointment: 1170.0>]>
<QuerySet [<Appointment: 1170.0>]>
<QuerySet [<Appointment: 1170.0>]>
